# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.2 k近傍法によるクラス分類とクラス分類モデルの推定性能の評価

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 数値データの特徴量を 説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # あやめの種類を目的変数 y とします

## k-NN によるクラス分類の実行

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, stratify=y, shuffle=True, random_state=3)

下はテキスト化していますが、shuffle=False とすると、ランダムに分割されるのではなく、下から test_size の数のサンプルがテストデータに、残りのサンプルがトレーニングデータになります。時系列データにおいて、時間的に古いサンプルをトレーニングデータに、新しいサンプルをテストデータとしたいときなどに利用します。

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, shuffle=False)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの特徴量の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの特徴量の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_x_train.mean()

In [ ]:
autoscaled_x_train.std()

In [ ]:
autoscaled_x_test.mean() # 必ずしも 0 にはなりません

In [ ]:
autoscaled_x_test.std() # 必ずしも 1 にはなりません

k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k_in_knn = 5 # k の値

In [ ]:
model = KNeighborsClassifier(n_neighbors=k_in_knn, metric='euclidean')

`metric` の種類 (第五回のクラスタリングのときの `metric` とは異なることに注意してください)
- 'euclidean' : ユークリッド距離
- 'manhattan' : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) をご覧ください 

In [ ]:
model.fit(autoscaled_x_train, y_train) # クラス分類モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータのクラスの推定

In [ ]:
model.predict(autoscaled_x_train) #トレーニングデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics

In [ ]:
set(y_train) # クラスの種類

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
class_types # 念のため確認

In [ ]:
metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 念のため確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 念のため確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

k の値を変えて、それぞれのクラス分類モデルの推定性能を評価してみましょう。

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

### 練習問題

データセット `virtual_equipment.csv` を読み込み、`state` をクラスの情報、2 つの温度 (`T1`, `T2`) を特徴量として、 k-NN でクラス分類してみましょう。なお、トレーニングデータ 50 個とテストデータ 50 個に、トレーニングデータとテストデータにおける各クラスの割合が均等になるように、かつランダムに分割してください。

### 仮想的な装置のデータセット (virtual_equipment.csv)
仮想的な装置において、時刻ごとの 2 つの温度の測定値 (`T1`, `T2`) とそのときの装置の状態 (`state`) が入ったデータセット。`state` の種類としては、normal (正常)、abnormal_1 (異常1)、abnormal_2 (異常2)、abnormal_3 (異常3) があります。

### 練習問題 コードの例

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('virtual_equipment.csv', index_col=0, header=0) # 仮想的な装置のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 0:2] # 数値データの特徴量を説明変数 x とします

In [ ]:
x # 念のため確認

In [ ]:
y = dataset.iloc[:, 2] # 装置の状態を目的変数 y とします

In [ ]:
y # 念のため確認

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, stratify=y, shuffle=True, random_state=3)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの特徴量の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの特徴量の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k_in_knn = 3 # k の値

In [ ]:
model = KNeighborsClassifier(n_neighbors=k_in_knn, metric='euclidean')

In [ ]:
model.fit(autoscaled_x_train, y_train) # クラス分類モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータのクラスの推定

In [ ]:
model.predict(autoscaled_x_train) #トレーニングデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics

In [ ]:
set(y_train) # クラスの種類

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
class_types # 念のため確認

In [ ]:
metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 念のため確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 念のため確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

k の値を変えて、それぞれのクラス分類モデルの推定性能を評価してみましょう。